In [7]:
from bs4 import BeautifulSoup
import requests as req
import re

# 네이버 웹툰 URL
res = req.get('https://comic.naver.com/webtoon/weekday.nhn')
soup = BeautifulSoup(res.text)

# {웹툰 이름 : 웹툰 번호} 딕셔너리
webtoon_dict = {}
for i in soup.select('.title'):
    title = i.get_text(strip=True)
    titleId = re.findall(r".*titleId=(\d+)&", i['href']).pop()
    webtoon_dict[title] = titleId
    
def get_soup(titleId):
    """ Returns soup of response that is user_in webtoon page
    :param titleId: (string) webtoon titleId number
    :return: (object) parsed page of response
    """
    res = req.get(f'https://comic.naver.com/webtoon/list.nhn?titleId={webtoon_dict[titleId]}')
    soup = BeautifulSoup(res.text)
    return soup

def webtoon_info_detail(user_in):
    """ Prints detail info of user_in webtoon
    :param user_in: user input webtoon title
    :return: None
    """
    print("\n", f"# 정보 ({user_in})")
    soup = get_soup(user_in)
    author = soup.select_one('.detail .wrt_nm').get_text(strip=True)
    desc = soup.select_one('.detail p').get_text('\n', strip=True)
    genre = soup.select_one('.detail .genre').get_text(strip=True)
    # genre_list = [i.strip() for i in soup.select_one('.detail .genre').get_text(strip=True).split(',')]
    print(f"작가: {author}\n"
          f"설명: {desc}\n"
          f"장르: {genre}\n")
    
def episode_list(user_in):
    """ Prints title, rating, date of user_in webtoon
    :param user_in: user input webtoon title
    :return: None
    """
    print("\n", f"# 에피소드목록 ({user_in})")
    soup = get_soup(user_in)
    title = soup.select('.title a')
    rating = soup.select('.rating_type strong')
    date = soup.select('.num')
    data = [dict(title=x.get_text(), rating=y.get_text(), date=z.get_text()) for x, y, z in zip(title, rating, date)]
    for d in data:
        print("{}\n  평점: {}, 날짜: {}".format(d['title'], d['rating'], d['date']))
        
def search_webtoon(user_in, choice):
    """ Search user input webtoon and call function
        This is sort of middleware
    :param user_in: user input webtoon title
    :param choice: user want what to get
    :return: None
    """
    find_data = [i for i in webtoon_dict if user_in in i]
    if choice == '1':
        print("\n", "# 정보를 가져올 웹툰선택")
    elif choice == '2':
        print("\n", "# 에피소드 목록을 가져올 웹툰선택")
    for n, title in enumerate(find_data):
        print(f'{n + 1}. {title}')
    print("0. 이전")
    webtoon_choice = int(input("> 웹툰 선택: "))
    if not webtoon_choice:
        return
    elif choice == '1':
        webtoon_info_detail(find_data[webtoon_choice - 1])
    elif choice == '2':
        episode_list(find_data[webtoon_choice - 1])
        
# 웹툰 크롤러 반복 실행
while True:
    print("## 웹툰 크롤러 ##")
    print("1. 정보 \n"
          "2. 에피소드 목록 \n"
          "0. 나가기 \n")
    choice = input("> 입력: ")
    if choice == '0':
        print("Thank you")
        break
    elif choice == '1' or '2':
        user_in = input("> 웹툰명을 입력하세요: ")
        search_webtoon(user_in, choice)
    else:
        print("!!! 숫자로 입력해 주세요 !!!")


## 웹툰 크롤러 ##
1. 정보 
2. 에피소드 목록 
0. 나가기 

> 입력: 0
Thank you


In [1]:
import requests
import re
from bs4 import BeautifulSoup


def start():
    while True:
        print('1. 정보\n2. 에피소드 목록\n0.나가기')
        select_menu = int(input('입력 : '))
        if select_menu == 0:
            print("thanks")
            break
        elif select_menu == 1:
            webtoon_info()
        elif select_menu == 2:
            webtoon_episode()
        else:
            print('===잘못된 선택입니다===')
            

def webtoon_info():
    result_dict = {}
    url = 'https://comic.naver.com/webtoon/weekday.nhn'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    select_webtoon = input('웹툰명을 입력하세요  : ')

    for idx, a_title in enumerate(soup.select(f'a.title[title*={select_webtoon}]')):
        title = a_title.get_text(strip=True)
        href = a_title['href']
        title_id = re.search(r'.*?titleId=([\d]+).*?', href).group(1)   ### group???
        result_dict[idx + 1] = dict(title=title, title_id=title_id)     ###title=title, title_Id=title_Id???

    while True:
        for key, value in result_dict.items():
            print(f'{key} : {value["title"]}')
        print('0 : 이전')
        select_webtoon = int(input('웹툰 선택 : '))

        try:
            if select_webtoon == 0:
                break
            else:
                title_id = result_dict[select_webtoon]['title_id']
                webtoon_detail(title_id)
        except KeyError:
            print('===잘못된 번호를 선택==')

start()

1. 정보
2. 에피소드 목록
0.나가기
입력 : 0
thanks


In [ ]:
import requests
import re
from bs4 import BeautifulSoup

def start():
    while True:
        print('1. 정보\n2. 에피소드 목록\n0.나가기')
        select_menu = int(input('입력 : '))
        if select_menu == 0:
            break
        elif select_menu == 1 or select_menu == 2:
            search_webtoon(select_menu)
        else:
            print('===잘못된 선택입니다===')

def get_html(*args: str) -> str:
    if args:
        url = f'https://comic.naver.com/webtoon/list.nhn?titleId={args}'
    else:
        url = 'https://comic.naver.com/webtoon/weekday.nhn'

    response = requests.get(url)
    return response.text

def search_webtoon(menu):
    result_dict = {}
    html = get_html()
    soup = BeautifulSoup(html, 'html.parser')

    select_webtoon = input('웹툰명을 입력하세요  : ')

    for a_title in soup.select(f'a.title[title*={select_webtoon}]'):
        title = a_title.get_text(strip=True)
        href = a_title['href']
        title_id = re.search(r'.*?titleId=([\d]+).*?', href).group(1)
        result_dict[title] = title_id

    while True:
        result_list = [key for key in result_dict.keys()]
        print(f'웹툰 목록 ({select_webtoon})')
        for idx, key in enumerate(result_list):
            print(f'{idx + 1} : {key}')
        print('0 : 이전')
        select_number = int(input('웹툰 선택 : '))

        try:
            if select_number == 0:
                break
            else:
                if menu == 1:
                    title_id = result_dict[result_list[select_number-1]]
                    webtoon_detail_info(title_id)
                    print('\n')
                else:
                    title_id = result_dict[result_list[select_number - 1]]
                    webtoon_episode_info(title_id)
                    print('\n')
        except IndexError:
            print('===잘못된 번호를 선택==')

def webtoon_detail_info(title_id: str):
    html = get_html(title_id)
    soup = BeautifulSoup(html, 'html.parser')

    div_detail = soup.select_one('div.detail')
    author = div_detail.select_one('span.wrt_nm').get_text(strip=True)
    description = div_detail.select_one('p').get_text('\n', strip=True)
    genre = div_detail.select_one('p.detail_info > span.genre').get_text(strip=True)

    print(f'작가 : {author}\n'
          f'설명 : {description}\n',
          f'장르: {genre}')

def webtoon_episode_info(title_id: str):

    html = get_html(title_id)
    soup = BeautifulSoup(html, 'html.parser')

    title_list = soup.select('table.viewList td.title > a')
    rating_list = soup.select('table.viewList td > div.rating_type > strong')
    date_list = soup.select('table.viewList td.num')

    for title, rating, date in zip(title_list, rating_list, date_list):
        print(f'{title.get_text(strip=True)}\n'
              f'평점 : {float(rating.get_text(strip=True))}, '
              f'날짜 : {date.get_text(strip=True)}')

start()